In [2]:
import numpy as np
import tensorflow as tf

In [103]:
with open('data/reviews.txt', 'r') as f:
    reviews = f.read()
with open('data/labels.txt', 'r') as f:
    labels = f.read()

In [104]:
# the reviews are delimited with newlines `\n`
reviews[:2000]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   \nstory of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is tu

In [105]:
# the labels are delimited with newlines `\n`
labels[:2000]

'positive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\

In [106]:
from string import punctuation
all_text = ''.join([c for c in reviews if c not in punctuation])
reviews = all_text.split('\n')

all_text = ' '.join(reviews)
words = all_text.split()

In [107]:
all_text[:2000]

'bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers   the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students  when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled          at           high  a classic line inspector i  m here to sack one of your teachers  student welcome to bromwell high  i expect that many adults of my age think that bromwell high is far fetched  what a pity that it isn  t    story of a man who has unnatural feelings for a pig  starts out with a opening scene that is a terrific example of absurd comedy  a formal orchestra audience is turned into an insane  violent m

In [108]:
words[:20]

['bromwell',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 'such']

In [109]:
# Create your dictionary that maps vocab words to integers here
from collections import Counter
counts = Counter(words)
print(type(counts))

vocab = sorted(counts, key=counts.get, reverse=True)

# Note that index start from 1
vocab_to_int = {word:i for i, word in enumerate(vocab, 1)}

<class 'collections.Counter'>


In [110]:
print("type of vocab", type(vocab))
print('total # of words: ', len(vocab_to_int))
print("first word:", vocab[0])
print("last word:", vocab[-1])
print("first word index:", vocab_to_int[vocab[0]])
print("last word index", vocab_to_int[vocab[-1]])

type of vocab <class 'list'>
total # of words:  74072
first word: the
last word: arminian
first word index: 1
last word index 74072


In [111]:
# Convert the reviews to integers, same shape as reviews list, but with integers
print('# of reviews before index: ', len(reviews))
reviews_ints = []
for review in reviews:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])
    
print('# of reviews after index: ', len(reviews_ints))

# of reviews before index:  25001
# of reviews after index:  25001


In [112]:
labels = labels.split('\n')
print(labels[:20])

['positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative']


In [113]:
# Convert labels to 1s and 0s for 'positive' and 'negative'
labels = np.array([1 if label == 'positive' else 0 for label in labels])
print("# of labels: ", len(labels))

# of labels:  25001


In [114]:
# Filter out that review with 0 length
non_zero_idx = [idx for idx, review in enumerate(reviews_ints) if len(review) != 0]
len(non_zero_idx)

25000

In [115]:
reviews_ints[-1]

[]

In [116]:
# Reconstruct reviews_ints and labels for only containing review with length > 0
reviews_ints = [reviews_ints[idx] for idx in non_zero_idx]
labels = np.array([labels[idx] for idx in non_zero_idx])

In [117]:
print('# of reviews:', len(reviews_ints))
print('# of labels:', len(labels))

# of reviews: 25000
# of labels: 25000


In [118]:
seq_len = 200

# The features created here are the data that we are going to train and test the network

# Create features with shape (len(reviews_ints), seq_len) and initialized with zeros
features = np.zeros((len(reviews_ints), seq_len), dtype=int)

# Create list holding the length for each review
lengths = []

# row is the review in forms of a list of integers
for i, row in enumerate(reviews_ints):
    
    # left padding
    features[i, -len(row):] = np.array(row)[:seq_len]
    
    # record the length of each review. This might be useful when we want to use sequence_length argument
    # of tf.nn.dynamic_rnn(...)
    lengths.append(len(row))

In [119]:
features[:10,:100]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 21614,   308,     6,
            3,  1050,   207,     8,  2140,    32,     1,   171,    57,
           15,    49,    81,  5848,    44,   382,   110,   140,    15,
         5207,    60,   154,     9,     1,  4993,  5859,   475,    71,
            5,   260,    12, 21614,   308,    13,  1982,     6,    74,
         2401],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     

In [120]:
print(labels.shape)
print(features.shape)

(25000,)
(25000, 200)


In [121]:
split_frac = 0.8
split = int(len(features) * split_frac)
train_x, val_x = features[:split, :], features[split:, :]
train_y, val_y = labels[:split], labels[split:]

split = int(len(val_x) * 0.5)
val_x, test_x = val_x[:split, :], val_x[split:, :]
val_y, test_y = val_y[:split], val_y[split:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


In [122]:
def get_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    
    # Only get full batches
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [123]:
# with graph.as_default():
    
#     def build_cell(lstm_units, keep_prob):
#         cell = tf.contrib.rnn.BasicLSTMCell(lstm_units)
#         drop = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob = keep_prob)
#         return drop

In [124]:
# hyperparameters
batch_size = 500 # this is mandatory, important, performance gain from 300 to 500

embedding_dim = 300
n_words = len(vocab_to_int) + 1

num_lstm_layer = 1
lstm_size = 256

learning_rate = 0.001

tf.reset_default_graph()

graph = tf.Graph()
with graph.as_default():
    
    # input placeholders
    inputs = tf.placeholder(tf.int32, shape=[None, None], name='inputs')
#     labels = tf.placeholder(tf.float32, shape=[None, None], name='labels')
    labels = tf.placeholder(tf.int32, shape=[None, None], name='labels')
    
#     seq_len = tf.placeholder(tf.int32, shape=[None], name='seq_len')
    keep_prob = tf.placeholder(tf.float32, name = "keep_prob")

    # embedding layer
    embedding = tf.Variable(tf.random_uniform((n_words, embedding_dim), -1, 1), name = 'embedding')
    embed = tf.nn.embedding_lookup(embedding, inputs)

    # LSTM layer
    def build_cell(lstm_units, keep_prob):
        cell = tf.contrib.rnn.BasicLSTMCell(lstm_units)
        drop = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob = keep_prob)
        return drop
    
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_lstm_layer)])
    initial_state = cell.zero_state(batch_size, tf.float32)

    outputs, final_state = tf.nn.dynamic_rnn(cell, 
                                             embed, 
#                                              sequence_length = seq_len,
                                             initial_state = initial_state)

    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    cost = tf.losses.mean_squared_error(labels, predictions)
#     cost = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=predictions)
#     entroy = tf.reduce_mean(cost)
#     optimizer = tf.train.AdamOptimizer(learning_rate).minimize(entroy)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

#     correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), tf.cast(tf.round(labels), tf.int32))
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [125]:
epochs = 10
# keep_prob = 0.5

with tf.Session(graph=graph) as sess:
        
        sess.run(tf.global_variables_initializer())
        iteration = 1
        
        for e in range(epochs):
            
            state = sess.run(initial_state)
            
            for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
#             for batch_features, batch_labels, batch_len in get_batches(x_train_p, train_label, x_train_len, batch_size):
#                 batch_len = np.array(batch_len)

#                 print(batch_features.shape)
#                 print(batch_labels.shape)
#                 print(batch_len.shape)
                
#                 feed = {inputs: batch_features,
#                         labels: batch_labels[:, None],   # change ths shape of y to [batch_size, 1]
# #                         seq_len: batch_len,
#                         keep_prob: 0.5,
#                         initial_state: state}
        
                feed = {inputs: x,
                        labels: y[:, None],   # change ths shape of y to [batch_size, 1]
#                         seq_len: batch_len,
                        keep_prob: 0.5,
                        initial_state: state}
        
#                 loss, state, _ = sess.run([entroy, final_state, optimizer], feed_dict=feed)
                loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
                if iteration%5==0:
                    print(loss)
                    print("Epoch: {}/{}".format(e, epochs),
                          "Iteration: {}".format(iteration),
                          "Train loss: {:.3f}".format(loss))
                iteration +=1

0.241309
Epoch: 0/10 Iteration: 5 Train loss: 0.241
0.246442
Epoch: 0/10 Iteration: 10 Train loss: 0.246
0.228689
Epoch: 0/10 Iteration: 15 Train loss: 0.229
0.198157
Epoch: 0/10 Iteration: 20 Train loss: 0.198
0.187036
Epoch: 0/10 Iteration: 25 Train loss: 0.187
0.204461
Epoch: 0/10 Iteration: 30 Train loss: 0.204
0.157914
Epoch: 0/10 Iteration: 35 Train loss: 0.158
0.180983
Epoch: 0/10 Iteration: 40 Train loss: 0.181
0.169045
Epoch: 1/10 Iteration: 45 Train loss: 0.169
0.176163
Epoch: 1/10 Iteration: 50 Train loss: 0.176
0.169173
Epoch: 1/10 Iteration: 55 Train loss: 0.169
0.177093
Epoch: 1/10 Iteration: 60 Train loss: 0.177
0.145949
Epoch: 1/10 Iteration: 65 Train loss: 0.146
0.212292
Epoch: 1/10 Iteration: 70 Train loss: 0.212
0.144274
Epoch: 1/10 Iteration: 75 Train loss: 0.144
0.160709
Epoch: 1/10 Iteration: 80 Train loss: 0.161
0.139427
Epoch: 2/10 Iteration: 85 Train loss: 0.139
0.148421
Epoch: 2/10 Iteration: 90 Train loss: 0.148
0.155358
Epoch: 2/10 Iteration: 95 Train loss: 